In [12]:
import coremltools
import numpy as np
import tensorflow as tf
import os
import tfcoreml
import h5py
import sympy

from tensorflow.keras.models import load_model

In [3]:
!pip show tensorflow

Name: tensorflow
Version: 2.2.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/beccahallam/Library/Python/3.7/lib/python/site-packages
Requires: scipy, keras-preprocessing, wheel, termcolor, six, tensorflow-estimator, astunparse, google-pasta, absl-py, opt-einsum, numpy, protobuf, tensorboard, wrapt, gast, grpcio, h5py
Required-by: tfcoreml, talos


## ML Model Conversion

From .h5 to .mlmodel for use in an iOS Application

### Import model

In [4]:
model_name = 'waste_classifier_model.h5'
directory = os.path.dirname(model_name)
model_file = os.path.join(directory, model_name)

In [5]:
model = load_model(model_file)

In [6]:
ip = model.input
op = model.output

print(ip)
print(op)

Tensor("conv2d_input:0", shape=(None, 300, 300, 3), dtype=float32)
Tensor("dense_2/Identity:0", shape=(None, 6), dtype=float32)


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43808)             0

In [28]:
print(model.accuracy)

AttributeError: 'Sequential' object has no attribute 'accuracy'

### Convert model

In [8]:
output_labels = ['cardboard',
                'glass',
                'metal',
                'paper',
                'plastic',
                'trash']

In [9]:
# get input, output node names for the TF graph from the Keras model
input_name = model.inputs[0].name.split(':')[0]
keras_output_node_name = model.outputs[0].name.split(':')[0]
graph_output_node_name = keras_output_node_name.split('/')[-1]

print(input_name)
print(keras_output_node_name)
print(graph_output_node_name)

conv2d_input
dense_2/Identity
Identity


In [24]:
waste_classifier = coremltools.converters.tensorflow.convert('waste_classifier_model.h5',
                                                             input_names=['image'],
                                                             output_names=['classLabelProbs','classLabel'],
                                                             class_labels=['0','1','2','3','4','5'])


0 assert nodes deleted
12 nodes deleted
0 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 12 nodes.
1 identity nodes deleted
6 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/20] Converting op type: 'Placeholder', name: 'conv2d_input', output_shape: (-1, 300, 300, 3).
[SSAConverter] [2/20] Converting op type: 'Const', name: 'sequential/flatten/Const', output_shape: (2,).
[SSAConverter] [3/20] Converting op type: 'Transpose', name: 'conv2d_input_to_nchw', output_shape: (-1, 3, 300, 300).
[SSAConverter] [4/20] Converting op type: 'Conv2D', name: 'sequential/conv2d/Conv2D', output_shape: (-1, 32, 300, 300).
[SSAConverter] [5/20] Converting op type: 'Relu', name: 'sequential/conv2d/Relu', output_shape: (-1, 32, 300, 300).
[SSAConverter] [6/20] Converting op type: 'MaxPool', name: 'sequential/max_pooling2d/MaxPool', output_shape: (-1, 32, -1, -1).
[SSAConverter] [7/20] Converting op type: 'Conv2D', name: 'sequential/conv2d_1/Conv2D'

In [25]:
print(waste_classifier)

input {
  name: "conv2d_input"
  type {
    multiArrayType {
      shape: 1
      shape: 300
      shape: 300
      shape: 3
      dataType: FLOAT32
      shapeRange {
        sizeRanges {
          lowerBound: 1
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 300
          upperBound: 300
        }
        sizeRanges {
          lowerBound: 300
          upperBound: 300
        }
        sizeRanges {
          lowerBound: 3
          upperBound: 3
        }
      }
    }
  }
}
output {
  name: "Identity"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "Identity"
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.4"
  }
}



### Provide information

In [26]:
waste_classifier.author = 'Rebecca Hallam'
waste_classifier.short_description = 'Waste classification with keras on the TrashNet dataset'


### Save as .mlmodel

In [27]:
waste_classifier.save('WasteClassifierV2.mlmodel')